# 📊 Génération Dataset ÉQUILIBRÉ avec TOUTES les constantes

**Améliorations :**
- ✅ Classes équilibrées (même nombre de ROUGE/JAUNE/VERT/GRIS)
- ✅ TOUTES les constantes (FC, FR, SpO2, TA sys/dia, Temp)
- ✅ Pathologies ciblées par gravité

## 1️⃣ Imports

In [1]:
import sys
from pathlib import Path


import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle 
import time 

project_root = Path.cwd().parent.parent  
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))


from src.llm.llm_factory import LLMFactory
from src.workflows.simulation_workflow import SimulationWorkflow

print("✅ Imports OK")

c:\Users\HP\Desktop\Master2\llm\Triage_urgence\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Imports OK


## 2️⃣ Charger CamemBERT-bio

In [2]:
print("🔧 Chargement CamemBERT-bio...")

model_name = "almanach/camembert-bio-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()

print("✅ Modèle chargé !")

🔧 Chargement CamemBERT-bio...


Loading weights: 100%|██████████| 197/197 [00:00<00:00, 763.18it/s, Materializing param=encoder.layer.11.output.dense.weight]              
CamembertModel LOAD REPORT from: almanach/camembert-bio-base
Key                             | Status     | 
--------------------------------+------------+-
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
lm_head.layer_norm.weight       | UNEXPECTED | 
pooler.dense.weight             | MISSING    | 
pooler.dense.bias               | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


✅ Modèle chargé !


## 3️⃣ Fonctions avec TOUTES les constantes

In [3]:
def encode_symptoms(symptoms: list) -> np.ndarray:
    """Encode symptômes (768 dim)."""
    if not symptoms:
        return np.zeros(768)
    
    text = ", ".join(symptoms)
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    return outputs.last_hidden_state[:, 0, :].numpy()[0]


def extract_features(patient_data: dict) -> np.ndarray:
    """Extrait TOUTES les features (776 dim)."""
    features = []
    
    # 1. Embeddings (768)
    symptoms = patient_data.get('symptomes', [])
    features.extend(encode_symptoms(symptoms))
    
    # 2.  TOUTES LES CONSTANTES (6)
    fc = patient_data.get('fc', 80)
    fr = patient_data.get('fr', 16)
    spo2 = patient_data.get('spo2', 98)
    ta_sys = patient_data.get('ta_systolique', 120)
    ta_dia = patient_data.get('ta_diastolique', 80)
    temp = patient_data.get('temperature', 37.0)
    
    # Normaliser
    features.extend([
        (fc - 70) / 30,
        (fr - 16) / 5,
        (spo2 - 95) / 5,
        (ta_sys - 120) / 20,
        (ta_dia - 80) / 10,
        (temp - 37) / 2
    ])
    
    # 3. Patient (2)
    age = patient_data.get('age', 50)
    sexe = patient_data.get('sexe', 'M')
    features.extend([(age - 50) / 25, 1 if sexe == 'M' else 0])
    
    return np.array(features)


def auto_label(patient_data: dict) -> str:
    """Labellisation automatique."""
    symptoms_text = " ".join(patient_data.get('symptomes', [])).lower()
    fc = patient_data.get('fc', 80)
    fr = patient_data.get('fr', 16)
    spo2 = patient_data.get('spo2', 98)
    temp = patient_data.get('temperature', 37.0)
    
    # ROUGE
    if spo2 < 90 or fr > 30:
        return "ROUGE"
    if fc > 130 or fc < 50:
        return "ROUGE"
    if "douleur thoracique" in symptoms_text or "poitrine" in symptoms_text:
        return "ROUGE"
    if "avc" in symptoms_text or "paralysie" in symptoms_text:
        return "ROUGE"
    if "hémorragie" in symptoms_text:
        return "ROUGE"
    
    # JAUNE
    if temp > 39:
        return "JAUNE"
    if "fracture" in symptoms_text:
        return "JAUNE"
    if spo2 < 95 or fr > 25:
        return "JAUNE"
    
    # VERT
    if temp > 38:
        return "VERT"
    if any(w in symptoms_text for w in ["gastro", "entorse", "infection"]):
        return "VERT"
    
    # GRIS
    if any(w in symptoms_text for w in ["certificat", "ordonnance", "rhume"]):
        return "GRIS"
    
    return "VERT"

print("✅ Fonctions créées")

✅ Fonctions créées


## 4️⃣ Pathologies par Gravité (ÉQUILIBRÉ)

In [4]:
# ⭐ Pathologies CIBLÉES pour équilibrer les classes

PATHOLOGIES = {
    "ROUGE": [
        "Homme de 65 ans avec infarctus du myocarde",
        "Femme de 58 ans avec AVC ischémique",
        "Homme de 72 ans avec hémorragie digestive",
        "Femme de 48 ans avec embolie pulmonaire",
        "Homme de 55 ans avec détresse respiratoire aiguë",
    ],
    "JAUNE": [
        "Femme de 35 ans avec fracture tibia-péroné",
        "Homme de 42 ans avec appendicite aiguë",
        "Femme de 28 ans avec colique néphrétique",
        "Homme de 50 ans avec pneumonie sévère",
        "Femme de 38 ans avec brûlure 2ème degré étendue",
    ],
    "VERT": [
        "Femme de 30 ans avec gastro-entérite",
        "Homme de 25 ans avec entorse cheville",
        "Femme de 45 ans avec infection urinaire",
        "Homme de 32 ans avec otite moyenne aiguë",
        "Femme de 28 ans avec conjonctivite",
    ],
    "GRIS": [
        "Homme de 22 ans pour certificat médical sport",
        "Femme de 40 ans pour renouvellement ordonnance",
        "Homme de 35 ans avec rhume léger",
        "Femme de 50 ans pour résultats analyses",
        "Homme de 28 ans avec petite coupure superficielle",
    ]
}

print("✅ Pathologies définies")
for gravity, paths in PATHOLOGIES.items():
    print(f"   {gravity} : {len(paths)} pathologies")

✅ Pathologies définies
   ROUGE : 5 pathologies
   JAUNE : 5 pathologies
   VERT : 5 pathologies
   GRIS : 5 pathologies


## 5️⃣ Génération ÉQUILIBRÉE

In [5]:
# Paramètres
CASES_PER_CLASS = 5  
DELAY = 3  # Secondes entre générations

print(f"🎲 Génération équilibrée")
print(f"   {CASES_PER_CLASS} cas × 4 classes = {CASES_PER_CLASS * 4} cas total")
print(f"   Délai : {DELAY}s entre chaque\n")

🎲 Génération équilibrée
   5 cas × 4 classes = 20 cas total
   Délai : 3s entre chaque



In [6]:
# Initialiser
llm = LLMFactory.create("mistral", "mistral-small-latest")
workflow = SimulationWorkflow(llm, max_turns=5)

X_list = []
y_list = []
metadata_list = []

# Générer par classe
for target_gravity in ["ROUGE", "JAUNE", "VERT", "GRIS"]:
    print(f"\n{'='*60}")
    print(f"🎯 Génération classe {target_gravity}")
    print(f"{'='*60}")
    
    pathologies = PATHOLOGIES[target_gravity]
    
    for i in tqdm(range(CASES_PER_CLASS)):
        try:
            # Choisir pathologie de cette classe
            pathology = pathologies[i % len(pathologies)]
            
            # Générer
            import io
            from contextlib import redirect_stdout
            
            with redirect_stdout(io.StringIO()):
                result = workflow.run_simulation(pathology=pathology)
            
            ml_data = workflow.export_for_ml()
            
            #  Extraire avec TOUTES les constantes
            features = extract_features(ml_data)
            
            # Label (déterministe ou automatique)
            label = target_gravity  # Forcer le label attendu
            
            X_list.append(features)
            y_list.append(label)
            metadata_list.append({
                'pathology': ml_data['pathology'],
                'age': ml_data['age'],
                'fc': ml_data.get('fc'),
                'fr': ml_data.get('fr'),
                'spo2': ml_data.get('spo2'),
                'temperature': ml_data.get('temperature')
            })
            
            workflow.reset()
            time.sleep(DELAY)
            
        except Exception as e:
            if "429" in str(e):
                print(f"\n⚠️ Rate limit. Pause 30s...")
                time.sleep(30)
            else:
                print(f"\n❌ Erreur : {e}")
                time.sleep(5)
    
    print(f"✅ {target_gravity} terminé : {len([y for y in y_list if y == target_gravity])} cas")

X = np.array(X_list)
y = np.array(y_list)

print(f"\n{'='*60}")
print(f"✅ GÉNÉRATION TERMINÉE")
print(f"   X shape : {X.shape}")
print(f"   y shape : {y.shape}")
print(f"{'='*60}")


🎯 Génération classe ROUGE


100%|██████████| 5/5 [01:34<00:00, 18.90s/it]


✅ ROUGE terminé : 5 cas

🎯 Génération classe JAUNE


100%|██████████| 5/5 [01:37<00:00, 19.57s/it]


✅ JAUNE terminé : 5 cas

🎯 Génération classe VERT


100%|██████████| 5/5 [01:34<00:00, 18.96s/it]


✅ VERT terminé : 5 cas

🎯 Génération classe GRIS


100%|██████████| 5/5 [01:37<00:00, 19.48s/it]

✅ GRIS terminé : 5 cas

✅ GÉNÉRATION TERMINÉE
   X shape : (20, 776)
   y shape : (20,)


## 6️⃣ Vérification Équilibre

In [7]:
from collections import Counter

counts = Counter(y)

print("📊 DISTRIBUTION (doit être équilibrée)")
print("="*60)
for label in ['ROUGE', 'JAUNE', 'VERT', 'GRIS']:
    count = counts.get(label, 0)
    pct = (count / len(y)) * 100
    bar = '█' * count
    print(f"   {label:6s} : {count:2d} ({pct:5.1f}%) {bar}")
print("="*60)

if len(set(counts.values())) == 1:
    print("✅ Classes parfaitement équilibrées !")
else:
    print("⚠️  Léger déséquilibre (normal si erreurs API)")

📊 DISTRIBUTION (doit être équilibrée)
   ROUGE  :  5 ( 25.0%) █████
   JAUNE  :  5 ( 25.0%) █████
   VERT   :  5 ( 25.0%) █████
   GRIS   :  5 ( 25.0%) █████
✅ Classes parfaitement équilibrées !


## 7️⃣ Vérification des Constantes

In [8]:
print(" VÉRIFICATION CONSTANTES")
print("="*60)

# Vérifier qu'on a bien toutes les constantes
sample = metadata_list[0]
print(f"Exemple cas 1 :")
print(f"   FC : {sample['fc']} bpm")
print(f"   FR : {sample['fr']} /min")
print(f"   SpO2 : {sample['spo2']}%")
print(f"   Temp : {sample['temperature']}°C")

# Indices dans features
print(f"\nIndices dans feature vector (776 dim) :")
print(f"   FC : index 768")
print(f"   FR : index 769 ")
print(f"   SpO2 : index 770")
print(f"   TA sys : index 771 ")
print(f"   TA dia : index 772 ")
print(f"   Temp : index 773")
print("="*60)

 VÉRIFICATION CONSTANTES
Exemple cas 1 :
   FC : 115 bpm
   FR : 22 /min
   SpO2 : 90%
   Temp : 36.8°C

Indices dans feature vector (776 dim) :
   FC : index 768
   FR : index 769 
   SpO2 : index 770
   TA sys : index 771 
   TA dia : index 772 
   Temp : index 773


## 8️⃣ Sauvegarder

In [9]:
data_dir = Path('../data')
data_dir.mkdir(exist_ok=True)

# Pickle
with open(data_dir / 'triage_dataset_balanced.pkl', 'wb') as f:
    pickle.dump({'X': X, 'y': y, 'metadata': metadata_list}, f)

print(f" Dataset sauvegardé : {data_dir / 'triage_dataset_balanced.pkl'}")

# CSV
df = pd.DataFrame(metadata_list)
df['gravity'] = y
df.to_csv(data_dir / 'triage_dataset_balanced.csv', index=False)

print(f" CSV sauvegardé : {data_dir / 'triage_dataset_balanced.csv'}")

 Dataset sauvegardé : ..\data\triage_dataset_balanced.pkl
 CSV sauvegardé : ..\data\triage_dataset_balanced.csv
